# Music Genre Classification Project

This notebook will use various Python libraries to perform build a machine learning model using audio file metadata.

## 1. Problem definition
> Given features extracted from an audio file, can we predict the genre that the audio belongs to

## 2. Data
The data came from the Free Music Archive developed by several people at École Polytechnique Fédérale de Lausann (EPFL) and Nanyang Technological University (NTU). 
The research paper can be found [here](https://arxiv.org/pdf/1612.01840.pdf) and the GitHub repository containing the project files can be found [here](https://github.com/mdeff/fma).

## 3. Evaluation
> If we can reach accuracy that is above the highest benchmark recorded in the research paper, the project is complete.

Here is the benchmarks taken from the FMA research paper:

| Feature set          | LR | kNN | SVM | MLP |
|----------------------|----|-----|-----|-----|
| 1 Chroma             | 44 | 44  | 48  | 49  |
| 2 Tonnetz            | 40 | 37  | 42  | 41  |
| 3 MFCC               | 58 | 55  | 61  | 53  |
| 4 Spec. centroid     | 42 | 45  | 46  | 48  |
| 5 Spec. bandwidth    | 41 | 45  | 44  | 45  |
| 6 Spec. contrast     | 51 | 50  | 54  | 53  |
| 7 Spec. rolloff      | 42 | 46  | 48  | 48  |
| 8 RMS energy         | 37 | 39  | 39  | 39  |
| 9 Zero-crossing rate | 42 | 45  | 45  | 46  |
| 3 + 6                | 60 | 55  | 63  | 54  |
| 3 + 6 + 4            | 60 | 55  | 63  | 53  |
| 1 to 9               | 61 | 52  | 63  | 58  |

The values in the table represent the accuracy % of a feature set and a given model.

The models are defined as:
* LR = Linear Regression with an _L<sup>2</sup>_ penalty
* kNN = k-nearest neighbours with k = 200
* SVM = support vector machines (SVM) with a radial basis function (RBF)
* MLP = multilayer perceptron (MLP) with 100 hidden neurons

## 4. Features
Here are the features that come with each audio file:
- [Chroma](https://en.wikipedia.org/wiki/Chroma_feature)
- [Tonnetz](https://en.wikipedia.org/wiki/Tonnetz)
- [Mel-frequency cepstral coefficients (MFCC)](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)
- [Spectral centroid](https://en.wikipedia.org/wiki/Centroid)
- [Spectral bandwidth](https://en.wikipedia.org/wiki/Bit_rate)
- [Spectral contrast](https://www.researchgate.net/publication/3968978_Music_type_classification_by_spectral_contrast_feature)
- [Spectral rolloff](https://www.dsprelated.com/freebooks/sasp/Spectral_Roll_Off.html)
- [RMS energy](http://replaygain.hydrogenaud.io/proposal/rms_energy.html)
- [Zero crossing-rate](https://en.wikipedia.org/wiki/Zero-crossing_rate)

## Preparing the tools

We will import all of the Python libraries we will use inside our project, including NumPy, Pandas, Matplotlib, and several Scikit-Learn ML models.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

import utils

%matplotlib inline

## Load the data

We will load the data using the `utils.py` utility file provided by the FMA repository.

In [3]:
features = utils.load("data/fma_metadata/features.csv")
tracks = utils.load("data/fma_metadata/tracks.csv")
echonest = utils.load('data/fma_metadata/echonest.csv')

## Pre-processing data

We will do our model testing with the small subset. Let us collect our set of training, validation, and testing data based on what was set from the FMA.

In [4]:
small_subset = tracks.index[tracks.set.subset == 'small']

In [5]:
small_tracks = tracks.loc[small_subset]
small_features = features.loc[small_subset]

In [6]:
train = small_tracks.index[small_tracks['set', 'split'] == 'training']
val = small_tracks.index[small_tracks['set', 'split'] == 'validation']
test = small_tracks.index[small_tracks['set', 'split'] == 'test']

We will create a helper function to collect the training, validation, and testing set for us.

In [7]:
def get_train_val_test_data(tracks, features, columns):
    """
    Generate training, validation, and testing data given `tracks` and `features`.
    """
    enc = LabelEncoder()
    labels = tracks.track.genre_top
    
    # Split into train, val, test
    X_train = features.loc[train, columns].values
    X_val = features.loc[val, columns].values
    X_test = features.loc[test, columns].values
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
        
    # Standardize features
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

## Modelling

Let's check which combination of features work best for testing. We will use the kNN model as our baseline model to make this determination (since to me the time it takes to train this model is very quick).

### Get combination of features

In [8]:
# Get list of all combinations of features
feature_sets = {}

for name in small_features.columns.levels[0]:
    feature_sets[name] = name

feature_sets.update({
    'mfcc/contrast': ['mfcc', 'spectral_contrast'],
    'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
    'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    'all_non-echonest': list(features.columns.levels[0])
})

feature_sets

{'chroma_cens': 'chroma_cens',
 'chroma_cqt': 'chroma_cqt',
 'chroma_stft': 'chroma_stft',
 'mfcc': 'mfcc',
 'rmse': 'rmse',
 'spectral_bandwidth': 'spectral_bandwidth',
 'spectral_centroid': 'spectral_centroid',
 'spectral_contrast': 'spectral_contrast',
 'spectral_rolloff': 'spectral_rolloff',
 'tonnetz': 'tonnetz',
 'zcr': 'zcr',
 'mfcc/contrast': ['mfcc', 'spectral_contrast'],
 'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
 'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
 'mfcc/contrast/chroma/centroid': ['mfcc',
  'spectral_contrast',
  'chroma_cens',
  'spectral_centroid'],
 'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc',
  'spectral_contrast',
  'chroma_cens',
  'spectral_centroid',
  'tonnetz'],
 'mfcc/contrast/chroma/centroid/zcr': ['mfcc',
  'spectral_contrast',
  'chroma_cens',
  'spectral_centroid',
  'zcr'],
 'all_non-echonest': ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'rmse',
  'spectral_bandwidth

### Prepare the model

In [318]:
np.random.seed(42)
model = KNeighborsClassifier(n_neighbors=200)

### Test different combination of features

We will use `cross_val_score` to have better clarity on which feature performs the best.

In [336]:
def getFeaturesAndLabelsGivenColumns(columns):
    enc = LabelEncoder()
    labels = small_tracks.track.genre_top

    X = small_features[columns].values
    y = enc.fit_transform(labels)

    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X)
    
    return X, y

In [338]:
for f_name, f_set in feature_sets.items():
    X, y = getFeaturesAndLabelsGivenColumns(f_set)
    cv_acc = cross_val_score(model, X, y, cv=5, scoring="accuracy")
    score = np.mean(cv_acc)
    print(f"Feature: {f_name}, Accuracy: {score:.2f}%")

Feature: chroma_cens, Accuracy: 0.25%
Feature: chroma_cqt, Accuracy: 0.27%
Feature: chroma_stft, Accuracy: 0.28%
Feature: mfcc, Accuracy: 0.41%
Feature: rmse, Accuracy: 0.24%
Feature: spectral_bandwidth, Accuracy: 0.31%
Feature: spectral_centroid, Accuracy: 0.32%
Feature: spectral_contrast, Accuracy: 0.35%
Feature: spectral_rolloff, Accuracy: 0.32%
Feature: tonnetz, Accuracy: 0.26%
Feature: zcr, Accuracy: 0.30%
Feature: mfcc/contrast, Accuracy: 0.42%
Feature: mfcc/contrast/chroma, Accuracy: 0.40%
Feature: mfcc/contrast/centroid, Accuracy: 0.42%
Feature: mfcc/contrast/chroma/centroid, Accuracy: 0.40%
Feature: mfcc/contrast/chroma/centroid/tonnetz, Accuracy: 0.39%
Feature: mfcc/contrast/chroma/centroid/zcr, Accuracy: 0.39%
Feature: all_non-echonest, Accuracy: 0.38%


As we see, we have two feature combinations that provide the highest accuracy: `["mfcc", "spectral_contrast"]` and `["mfcc", "spectral_contrast", "spectral_centroid"]`. For now we will use `mfcc/contrast` but we will further test with adding `spectral_centroid` to see the performance results.

### Test different models

Now we will test different models on the features to see which model give the best performance. 

In [345]:
ml_models = {
    "GaussianNB": GaussianNB(),
    "RandomForestClassifier": RandomForestClassifier(),
    "kNN": KNeighborsClassifier(n_neighbors=200),
    "SVC-linear": SVC(kernel="linear"),
    "SVC-poly": SVC(kernel="poly", degree=1),
    "SVC-rbf": SVC(kernel="rbf"),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000),
    "AdaBoost": AdaBoostClassifier(n_estimators=10)
}

fset = ["mfcc", "spectral_contrast"]
X, y = getFeaturesAndLabelsGivenColumns(fset)

for model_name, ml_model in ml_models.items():
    cv_acc = cross_val_score(ml_model, X, y, cv=5, scoring="accuracy")
    score = np.mean(cv_acc)
    print(f"Model: {model_name}, Accuracy: {score}")

Model: GaussianNB, Accuracy: 0.4085
Model: RandomForestClassifier, Accuracy: 0.499625
Model: kNN, Accuracy: 0.41987499999999994
Model: SVC-linear, Accuracy: 0.496125
Model: SVC-poly, Accuracy: 0.518375
Model: SVC-rbf, Accuracy: 0.536875
Model: MLP, Accuracy: 0.460375
Model: AdaBoost, Accuracy: 0.372375


According to the benchmark results above, it seems that our SVC model with radial basis functions performed the best with a 53.7% prediction rate! We will be selecting this model for performance.

Now we have our model, how can we improve our performance accuracy? We should perform hyperparameter tuning to see what different settings different results bring.

### Hyperparameter Tuning

According to [scikit-learn.org](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html), there are two parameters that can be tuned for our SVC model: `gamma` and `C`. Since our parameter choices are small, we can perform `GridSearchCV` to figure out which parameters performs the best.

In [346]:
# Set our model to SVC
np.random.seed(42)
model = SVC(kernel="rbf")

In [381]:
grid = {
    "C": [10**-4, 10**-2, 10**0, 10**2, 10**4],
    "gamma": [10**-2, 10**-1, 10**0, 10**1, 10**2]
}

# Setup GridSearchCV
tuned_model = GridSearchCV(estimator=model, 
                            param_grid=grid,
                           n_jobs=10, 
                           cv=5,
                           verbose=2)

# Prepare our test data
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)

In [382]:
tuned_model.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done 125 out of 125 | elapsed:  5.9min finished


GridSearchCV(cv=5, estimator=SVC(C=10, gamma=0.1), n_jobs=10,
             param_grid={'C': [0.0001, 0.01, 1, 100, 10000],
                         'gamma': [0.01, 0.1, 1, 10, 100]},
             verbose=2)

Let's check to see which parameters returned the best score.

In [383]:
tuned_model.best_params_

{'C': 1, 'gamma': 0.01}

Now that we've tuned the model and got these params, let's check to see if we actually got a better result.

In [388]:
model = SVC(kernel="rbf", C=1.0, gamma=0.01)
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

In [389]:
score

0.47625

As we see, tuning the hyperparameters returned a score _much_ less than the default score. In this case, it doesn't seem like changing the hyperparameters gives us a better result. 

Another suggestion we can do is tune the parameters for the next best model: SVC with `kernel=poly`.

In [399]:
grid = {
    "degree": [1, 2, 3, 5],
}

model = SVC(kernel="poly")
# Setup GridSearchCV
tuned_model = RandomizedSearchCV(estimator=model, 
                            param_distributions=grid,
                           n_iter=10, 
                           cv=5,
                           verbose=2)

# Prepare our test data
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)

In [ ]:
tuned_model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] gamma=100, degree=1, C=10000 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [392]:
tuned_model.best_params_

{'degree': 1}

In [397]:
model = SVC(kernel="poly", degree=1)
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

In [398]:
score

0.41875

In [10]:
grid = {"n_estimators": [10, 100, 200, 500, 1000, 1200],
       "max_depth": [None, 5, 10, 20, 30],
       "max_features": ["auto", "sqrt"],
       "min_samples_split": [2, 4, 6],
       "min_samples_leaf": [1, 2, 4]}

fset = ["mfcc", "spectral_contrast"]

model = RandomForestClassifier()

# Setup RandomizedSearchCV
tuned_model = RandomizedSearchCV(estimator=model, 
                            param_distributions=grid,
                           n_iter=10, # num of models to try
                           cv=5,
                           verbose=2)

X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)

In [11]:
tuned_model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=  19.2s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.2s remaining:    0.0s


[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=  19.2s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=  18.9s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=  19.0s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=4, max_features=sqrt, max_depth=5, total=  18.9s
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30 
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=30, total=  18.9s
[CV] n_estimators=500, min_samples_split=2, min_samp

[CV]  n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10, total=  31.8s
[CV] n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10, total=  31.5s
[CV] n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10, total=  31.5s
[CV] n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10 
[CV]  n_estimators=1000, min_samples_split=4, min_samples_leaf=2, max_features=auto, max_depth=10, total=  31.3s
[CV] n_estimators=500, min_samples_split=6, min_samples_leaf=1, max_features=auto, max_depth=5 
[CV]  n_estimators=500, min_samples_split=6, min_samples_leaf=1, max_features=auto, max_depth=5, total=   9.3s
[CV] n_estimators=500, min_samples_split=6, min

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 18.7min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [10, 100, 200, 500,
                                                         1000, 1200]},
                   verbose=2)

In [12]:
tuned_model.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 20}

In [13]:
tuned_model.score(X_test, y_test)

0.45875

In [26]:
model = SVC(kernel="rbf")
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

In [27]:
score

0.47875

This is an example of one of the downfalls of searching for hyperparameters to tune. Sometimes the results come out worse than the default baseline models, and that's okay. The baseline model could be enough to receive the best predictions.

Let's add `spectral_centroid` to see if we receive higher accuracy.

In [28]:
fset = ['mfcc', 'spectral_contrast', 'spectral_centroid']
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(small_tracks, small_features, fset)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
score

0.49

## Evaluation

Now that we've received our baseline model, we can now do predictions on the medium dataset to compare our benchmarks to the research papers.

### Prepare our data

In [39]:
medium_subset = tracks.index[tracks.set.subset == 'medium']
medium_tracks = tracks.loc[medium_subset]
medium_features = features.loc[medium_subset]

train = medium_tracks.index[medium_tracks['set', 'split'] == 'training']
val = medium_tracks.index[medium_tracks['set', 'split'] == 'validation']
test = medium_tracks.index[medium_tracks['set', 'split'] == 'test']

fset = ['mfcc', 'spectral_contrast', 'spectral_centroid']

X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test_data(medium_tracks, medium_features, fset)

### Prepare our model

In [47]:
model = SVC(kernel="rbf")

### Fit the data to the model

In [48]:
model.fit(X_train, y_train)

SVC()

### Evaluate the model

In [49]:
model.score(X_test, y_test)

0.7157360406091371

## Discussion

We concluded the project with a prediction of 71.6%! According to our table listed above, the largest prediction we gained from the research paper was 63%. 

**Note:** This is just a quick introductory project to get myself used to testing different features, different models, and different hyperparameters. There's a lot of deeper studying of the dataset to do if I want to achieve better results. However, I will leave this work to another project I plan to move to production. This project is just to showcase my knowledge on the aforementioned subjects and other projects will showcase more focused, productive work.

Yanique Andre